In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import random
from scipy.linalg import fractional_matrix_power


# Coupling

In [4]:
def maximal_coupling(mu_x, mu_y, prv, qrv, log_p, log_q, tuning):
    x_prop = prv()
    
    if np.log(random.uniform(0, 1)) <= log_q(x_prop) - log_p(x_prop):
        return x_prop, x_prop
    else:
        y_prop = qrv()
        while np.log(random.uniform(0, 1)) <= log_p(y_prop) - log_q(y_prop):
            y_prop = qrv()
            
        return x_prop, y_prop


In [5]:
# assumes square covariance matrix (for fully metropolis)
def reflective_coupling1(mu_x, mu_y, prv, qrv, log_p, log_q, tuning):
    Delta = np.matmul(fractional_matrix_power(tuning, -1/2), mu_x - mu_y)
    
    e = Delta / np.sqrt(np.sum(np.power(Delta, 2)))
    mu = np.zeros(len(mu_x))
    sig = np.eye(len(mu_x))
    x_dot = scipy.stats.multivariate_normal.rvs(mu, sig)
    y_dot = scipy.stats.multivariate_normal.rvs(mu, sig)
    if np.log(random.uniform(0, 1)) <= scipy.stats.multivariate_normal.logpdf(x_dot + Delta, mu, sig) - scipy.stats.multivariate_normal.logpdf(x_dot, mu, sig):
        y_dot = x_dot + Delta
    else:
        y_dot = x_dot - 2 * np.dot(e, x_dot) * e
        
    return np.matmul(fractional_matrix_power(tuning, 1/2), x_dot) + mu_x, np.matmul(fractional_matrix_power(tuning, 1/2), y_dot) + mu_y
        

In [6]:
# tuning is an integer (for metropolis gibbs)
def reflective_coupling2(mu_x, mu_y, prv, qrv, log_p, log_q, tuning):
    Delta = 1 / np.sqrt(tuning) * (mu_x - mu_y)
    e = 1

    x_dot = scipy.stats.norm.rvs(0, 1)
    if np.log(random.uniform(0, 1)) <= scipy.stats.norm.logpdf(x_dot + Delta, 0, 1) - scipy.stats.norm.logpdf(x_dot, 0, 1):
        y_dot = x_dot + Delta
        
        return np.sqrt(tuning) * x_dot + mu_x, np.sqrt(tuning) * y_dot + mu_y
    else:
        y_dot = x_dot - 2 * x_dot
        
        if len(np.array([Delta]).ravel()) > 1:
            y_dot = x_dot - 2 * np.dot(e, x_dot) * e

        return np.sqrt(tuning) * x_dot + mu_x, np.sqrt(tuning) * y_dot + mu_y
        

# Accept/Reject

In [7]:
def ar(prev, prop, crit, log_pdf):
    log_r = log_pdf(prop) - log_pdf(prev)
    if crit <= log_r:
        return prop
    else:
        return prev

# Convergence Fucntion

In [8]:
def equality(px, py):
    if all (px == py):
        return True
    else:
        return False

In [9]:
def adaptive_converge1(px, py, Gx, Gy, sx, sy, eps, thresh):
    lx = Gx + eps
    ly = Gy + eps
    if all(px == py) and all(np.ravel(np.abs(np.exp(2 * sx) * lx - np.exp(2 * sy) * ly)) < thresh):
        return True
    else:
        return False

In [10]:
def adaptive_converge2(px, py, Gx, Gy, sx, sy, eps, thresh):
    lx = Gx + eps
    ly = Gy + eps
    if all(px == py) and all(np.ravel(np.abs((np.exp(2 * sx) * lx) / (np.exp(2 * sy) * ly))) < (1 + thresh)) and all(np.ravel(np.abs((np.exp(2 * sx) * lx) / (np.exp(2 * sy) * ly))) > (1 - thresh)):
        return True
    else:
        return False

# Log Density

In [11]:
def log_bb(data, params):
    if any(params <= 0) or any(params[2:] > 1):
        return -float("inf")
    else:
        return -len(data[0]) * np.log(scipy.special.beta(params[0], params[1])) + np.sum((data[0] + params[0] - 1) * np.log(params[2:])) + np.sum((data[1] - data[0] + params[1] - 1) * np.log(1 - params[2:]))
        

In [12]:
def log_pg(data, params):
    if any(params <= 0):
        return -float("inf")
    else:
        n = len(data)
        return params[0] * n * np.log(params[1]) - n * scipy.special.gammaln(params[0]) + np.sum((params[0] + data - 1) * np.log(params[2:])) - np.sum((params[1] + 1) * params[2:])


In [13]:
def log_ab(data, params):
    if any(params[-1] <= 0):
        return -float("inf")
    else:
        return -15 * np.log(scipy.special.beta(params[-1][0], params[-1][1])) + params[-1][0] * np.sum(np.log(params[0])) + params[-1][1] * np.sum(np.log(1 - params[0]))

# Unbiased MCMC

In [9]:
def coupled_metropolis1(data, x_init, y_init, converge_fun, coupling_fun, logpdf_fun, tuning, k = 100, lag = 1, maximum_iteration = 10000, verbose = True):
    
    params_x = [np.array(x_init)]
    params_y = [np.array(y_init)]
    
    iteration = 0
    
    logpdf = lambda z: logpdf_fun(data, z)
    
    while not converge_fun(params_x[-1], params_y[-1]):
        if lag <= iteration:
            prev_x = params_x[-1]
            prev_y = params_y[-1]
            
            x_rv = lambda: scipy.stats.multivariate_normal.rvs(prev_x, tuning)
            y_rv = lambda: scipy.stats.multivariate_normal.rvs(prev_y, tuning)
            log_x = lambda z: scipy.stats.multivariate_normal.logpdf(z, prev_x, tuning)
            log_y = lambda z: scipy.stats.multivariate_normal.logpdf(z, prev_y, tuning)
            
            x_prop, y_prop = coupling_fun(prev_x, prev_y, x_rv, y_rv, log_x, log_y, tuning) 
                    
            log_u = np.log(random.uniform(0, 1))
            params_x.append(ar(params_x[-1], x_prop, log_u, logpdf))
            params_y.append(ar(params_y[-1], y_prop, log_u, logpdf))
        else:
            x_prop = scipy.stats.multivariate_normal.rvs(params_x[-1], tuning)
            log_u = np.log(random.uniform(0, 1))
            params_x.append(ar(params_x[-1], x_prop, log_u, logpdf))
        
        iteration += 1
        
        if iteration == maximum_iteration:
            break

    if verbose:
        print(iteration)
    
    if iteration != maximum_iteration:
        for i in range(k):
            x_prop = scipy.stats.multivariate_normal.rvs(params_x[-1], tuning)
            logu = np.log(random.uniform(0, 1))
            temp = ar(params_x[-1], x_prop, logu, logpdf)
            params_x.append(temp)
            params_y.append(temp)
        
    return params_x, params_y, iteration
    

In [16]:
def coupled_metropolis2(data, x_init, y_init, converge_fun, coupling_fun, logpdf_fun, tuning, k = 100, lag = 1, maximum_iteration = 10000, verbose = True):

    params_x = [np.array(x_init)]
    params_y = [np.array(y_init)]
    
    iteration = 0
    
    while not converge_fun(params_x[-1], params_y[-1]):
        if lag <= iteration:
            x_prop = np.zeros(len(params_x[-1]))
            y_prop = np.zeros(len(params_x[-1]))

            for j in range(len(x_prop)):
                prev_x = params_x[-1][j]
                prev_y = params_y[-1][j]
                
                x_rv = lambda: scipy.stats.norm.rvs(prev_x, np.sqrt(tuning[j]))
                y_rv = lambda: scipy.stats.norm.rvs(prev_y, np.sqrt(tuning[j]))
                log_x = lambda z: scipy.stats.norm.logpdf(z, prev_x, np.sqrt(tuning[j]))
                log_y = lambda z: scipy.stats.norm.logpdf(z, prev_y, np.sqrt(tuning[j]))
                
                temp_x, temp_y = coupling_fun(prev_x, prev_y, x_rv, y_rv, log_x, log_y, tuning[j])
                
                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(prev_x, temp_x, log_u, lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:]))))
                y_prop[j] = ar(prev_y, temp_y, log_u, lambda z: logpdf_fun(data, np.hstack((y_prop[:j], z, params_y[-1][j + 1:]))))
            
            params_x.append(x_prop)
            params_y.append(y_prop)

        else:
            x_prop = np.zeros(len(params_x[-1]))
            
            for j in range(len(x_prop)):
                temp_x = scipy.stats.norm.rvs(params_x[-1][j], np.sqrt(tuning[j]))
                
                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(params_x[-1][j], temp_x, log_u, lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:]))))

            params_x.append(x_prop)
            
        iteration += 1
        if iteration == maximum_iteration:
            break

    if verbose:
        print(iteration)

    if iteration != maximum_iteration:
        for i in range(k):
            x_prop = np.zeros(len(params_x[-1]))
            
            for j in range(len(x_prop)):
                temp_x = scipy.stats.norm.rvs(params_x[-1][j], np.sqrt(tuning[j]))
                
                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(params_x[-1][j], temp_x, log_u, lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:]))))
    
            params_x.append(x_prop)
            params_y.append(x_prop)
        
    return params_x, params_y, iteration
    

In [11]:
def coupled_gibbs_rat(data, x_init, y_init, converge_fun, coupling_fun, logpdf_fun, tuning, k = 100, lag = 1, maximum_iteration = 10000, verbose = True):

    params_x = [x_init]
    params_y = [y_init]
    
    iteration = 0

    while not converge_fun(params_x[-1][-1], params_y[-1][-1]):
        if lag <= iteration:
            temp_x = np.zeros(len(x_init), dtype = 'object')
            temp_y = np.zeros(len(x_init), dtype = 'object')

            x_prop = np.zeros(len(params_x[-1][0]))
            y_prop = np.zeros(len(params_y[-1][0]))
            
            for j in range(15):
                prev_x = params_x[-1][0][j]
                prev_y = params_y[-1][0][j]
                
                x_rv = lambda: scipy.stats.beta.rvs(data[0][j] + params_x[-1][-1][0], data[1][j] - data[0][j] + params_x[-1][-1][1])
                y_rv = lambda: scipy.stats.beta.rvs(data[0][j] + params_y[-1][-1][0], data[1][j] - data[0][j] + params_y[-1][-1][1])
                log_x = lambda z: scipy.stats.beta.logpdf(z, data[0][j] + params_x[-1][-1][0], data[1][j] - data[0][j] + params_x[-1][-1][1])
                log_y = lambda z: scipy.stats.beta.logpdf(z, data[0][j] + params_y[-1][-1][0], data[1][j] - data[0][j] + params_y[-1][-1][1])
                
                x_prop[j], y_prop[j] = coupling_fun(prev_x, prev_y, x_rv, y_rv, log_x, log_y, 0)
            
            temp_x[0] = x_prop
            temp_y[0] = y_prop
                
            x_prop = np.zeros(len(params_x[-1][-1]))
            y_prop = np.zeros(len(params_x[-1][-1]))
            
            for j in range(len(x_prop)):
                prev_x = params_x[-1][-1][j]
                prev_y = params_y[-1][-1][j]

                x_rv = lambda: scipy.stats.norm.rvs(prev_x, np.sqrt(tuning[j]))
                y_rv = lambda: scipy.stats.norm.rvs(prev_y, np.sqrt(tuning[j]))
                log_x = lambda z: scipy.stats.norm.logpdf(z, prev_x, np.sqrt(tuning[j]))
                log_y = lambda z: scipy.stats.norm.logpdf(z, prev_y, np.sqrt(tuning[j]))

                temp_x_prop, temp_y_prop = coupling_fun(prev_x, prev_y, x_rv, y_rv, log_x, log_y, tuning[j])
                
                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(prev_x, temp_x_prop, log_u, lambda z: logpdf_fun(data, np.array(list(temp_x[:-1]) + [np.hstack((x_prop[:j], z, params_x[-1][-1][j + 1:]))], dtype = 'object')))
                y_prop[j] = ar(prev_y, temp_y_prop, log_u, lambda z: logpdf_fun(data, np.array(list(temp_y[:-1]) + [np.hstack((y_prop[:j], z, params_y[-1][-1][j + 1:]))], dtype = 'object')))
            
            temp_x[-1] = x_prop
            temp_y[-1] = y_prop
            
            params_x.append(temp_x)
            params_y.append(temp_y)

        else:
            temp_x = np.zeros(len(x_init), dtype = 'object')

            x_prop = np.zeros(len(params_x[-1][0])) # 15
            
            for j in range(15):
                x_prop[j] = scipy.stats.beta.rvs(data[0][j] + params_x[-1][-1][0], data[1][j] - data[0][j] + params_x[-1][-1][1])
            
            temp_x[0] = x_prop
            
            x_prop = np.zeros(len(params_x[-1][-1]))

            for j in range(len(x_prop)):
                prev_x = params_x[-1][-1][j]

                temp_x_prop = scipy.stats.norm.rvs(prev_x, np.sqrt(tuning[j]))

                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(prev_x, temp_x_prop, log_u, lambda z: logpdf_fun(data, np.array(list(temp_x[:-1]) + [np.hstack((x_prop[:j], z, params_x[-1][-1][j + 1:]))], dtype = 'object')))
            
            temp_x[-1] = x_prop
            
            params_x.append(temp_x)            
            
        iteration += 1
        if iteration == maximum_iteration:
            break

    if verbose:
        print(iteration)

    if iteration != maximum_iteration:
        for i in range(k):
            temp_x = np.zeros(len(x_init), dtype = 'object')

            x_prop = np.zeros(len(params_x[-1][0])) # 15
            
            for j in range(15):
                x_prop[j] = scipy.stats.beta.rvs(data[0][j] + params_x[-1][-1][0], data[1][j] - data[0][j] + params_x[-1][-1][1])
            
            temp_x[0] = x_prop
            
            x_prop = np.zeros(len(params_x[-1][-1]))

            for j in range(len(x_prop)):
                prev_x = params_x[-1][-1][j]

                temp_x_prop = scipy.stats.norm.rvs(prev_x, np.sqrt(tuning[j]))

                log_u = np.log(random.uniform(0, 1))
                x_prop[j] = ar(prev_x, temp_x_prop, log_u, lambda z: logpdf_fun(data, np.array(list(temp_x[:-1]) + [np.hstack((x_prop[:j], z, params_x[-1][-1][j + 1:]))], dtype = 'object')))
            
            temp_x[-1] = x_prop
            
            params_x.append(temp_x) 
            params_y.append(temp_x) 

    return params_x, params_y, iteration
    

# Adaptive MCMC

In [12]:
# For fully Metropolis
def adaptive_metro1(data, init, logpdf_fun, eps = 0.01, max_iteration = 100000):
    params = [np.array(init)]
    accept = 1
    total = 1
    
    iteration = 0
    mu = params[-1]
    
    Gamma = np.zeros((len(params[-1]), len(params[-1])))
                     
    sigma = 0.7

    n = 0
    gamma = 1
    
    logpdf = lambda z: logpdf_fun(data, z)
    
    for _ in range(max_iteration):
        lamb = Gamma + eps * np.eye(len(Gamma))

        prop = np.random.multivariate_normal(mean = params[-1], cov = np.exp(2 * sigma) * lamb)

        log_u = np.log(random.uniform(0, 1))

        log_r = min(0, logpdf(prop) - logpdf(params[-1]))

        if log_u <= log_r:
            accept += 1
            params.append(prop)
        else:
            params.append(params[-1])

        n += 1

        m = params[-1] - mu
        Gamma = Gamma + (np.matmul(np.atleast_2d(m).T, np.atleast_2d(m)) - Gamma) / n

        mu = mu + (params[-1] - mu) / n

        gamma = np.sqrt(1/n)
        sigma = sigma + gamma * (np.exp(log_r) - 0.3)
    
    return params, mu, Gamma, sigma, iteration, accept/total
    

In [13]:
# For Metropolis within Gibbs

def adaptive_metro2(data, init, logpdf_fun, eps = 0.01, max_iteration = 10000):
    params = [np.array(init)]
    accept = 1
    total = 1
    
    iteration = 0
    mu = params[-1]
    
    Gamma = np.zeros(len(params[-1]))
    sigma = np.ones(len(params[-1])) * 0.7

    n = 0
    gamma = 1
    
    for _ in range(max_iteration):
        temp = np.zeros(len(params[-1]))
        log_r = np.zeros(len(params[-1]))
        
        for j in range(len(params[-1])):
            lamb = Gamma[j] + eps
            logpdf = lambda z: logpdf_fun(data, np.hstack((temp[:j], z, params[-1][j + 1:])))
            prop = np.random.normal(params[-1][j], np.sqrt(np.exp(2 * sigma[j]) * lamb))

            log_u = np.log(random.uniform(0, 1))
            log_r[j] = min(0, logpdf(prop) - logpdf(params[-1][j]))


            if log_u <= log_r[j]:
                accept += 1
                temp[j] = prop
            else:
                temp[j] = params[-1][j]

        params.append(temp)

        total += len(params[-1])
        n += 1

        m = params[-1] - mu
        Gamma = Gamma + (np.power(m, 2) - Gamma) / n
        
        mu = mu + (params[-1] - mu)/n

        gamma = np.sqrt(1/n)
        sigma = sigma + gamma * (np.exp(log_r) - 0.3)

        iteration += 1

        if iteration == 100000:
            break

        
    print(iteration)
    
    return params, mu, Gamma, sigma, iteration, accept/total
    

# "Controlled Bias" MCMC

In [14]:
def coupled_adaptive_metropolis1(data, x_init, y_init, converge_fun, coupling_fun, logpdf_fun, eps = 0.01, k = 100, lag = 1, maximum_iteration = 10000):
    params_x = [np.array(x_init)]
    params_y = [np.array(y_init)]

    mu_x = params_x[-1]
    Gamma_x = np.zeros((len(params_x[-1]), len(params_x[-1])))
    sigma_x = 0.7
    nx = 0
    gamma_x = 1

    mu_y = params_y[-1]
    Gamma_y = np.zeros((len(params_y[-1]), len(params_y[-1])))
    sigma_y = 0.7
    ny = 0
    gamma_y = 1
    
    iteration = 0
    
    logpdf = lambda z: logpdf_fun(data, z)
    
    while not converge_fun(params_x[-1], params_y[-1], Gamma_x, Gamma_y, sigma_x, sigma_y, eps, 1):
        if lag <= iteration:
            lamb_x = Gamma_x + eps * np.eye(len(Gamma_x))
            lamb_y = Gamma_y + eps * np.eye(len(Gamma_y))
            tuning_x = np.exp(2 * sigma_x) * lamb_x
            tuning_y = np.exp(2 * sigma_y) * lamb_y

            prev_x = params_x[-1]
            prev_y = params_y[-1]
            
            x_rv = lambda: scipy.stats.multivariate_normal.rvs(prev_x, tuning_x)
            y_rv = lambda: scipy.stats.multivariate_normal.rvs(prev_y, tuning_y)
            log_x = lambda z: scipy.stats.multivariate_normal.logpdf(z, prev_x, tuning_x)
            log_y = lambda z: scipy.stats.multivariate_normal.logpdf(z, prev_y, tuning_y)
            
            x_prop, y_prop = coupling_fun(x_rv, y_rv, log_x, log_y) 

            log_u = np.log(random.uniform(0, 1))

            
            log_rx = min(0, logpdf(x_prop) - logpdf(prev_x))
            log_ry = min(0, logpdf(y_prop) - logpdf(prev_y))
    
            if log_u <= log_rx:
                params_x.append(x_prop)
            else:
                params_x.append(prev_x)
    
            if log_u <= log_ry:
                params_y.append(y_prop)
            else:
                params_y.append(prev_y)
            
            nx += 1
            ny += 1
            
            m_x = params_x[-1] - mu_x
            Gamma_x = Gamma_x + (np.matmul(np.atleast_2d(m_x).T, np.atleast_2d(m_x)) - Gamma_x) / nx
    
            mu_x = mu_x + (params_x[-1] - mu_x) / nx
    
            gamma_x = np.sqrt(1/nx)
            sigma_x = sigma_x + gamma_x * (np.exp(log_rx) - 0.3)

            m_y = params_y[-1] - mu_y
            Gamma_y = Gamma_y + (np.matmul(np.atleast_2d(m_y).T, np.atleast_2d(m_y)) - Gamma_y) / ny
    
            mu_y = mu_y + (params_y[-1] - mu_y) / ny
    
            gamma_y = np.sqrt(1/ny)
            sigma_y = sigma_y + gamma_y * (np.exp(log_ry) - 0.3)

    
        else:
            lamb_x = Gamma_x + eps * np.eye(len(Gamma_x))
            tuning_x = np.exp(2 * sigma_x) * lamb_x

            prev_x = params_x[-1]
            
            x_prop = scipy.stats.multivariate_normal.rvs(prev_x, tuning_x)
            log_u = np.log(random.uniform(0, 1))

            
            log_rx = min(0, logpdf(x_prop) - logpdf(prev_x))
    
            if log_u <= log_rx:
                params_x.append(x_prop)
            else:
                params_x.append(prev_x)

            nx += 1
            
            m_x = params_x[-1] - mu_x
            Gamma_x = Gamma_x + (np.matmul(np.atleast_2d(m_x).T, np.atleast_2d(m_x)) - Gamma_x) / nx
    
            mu_x = mu_x + (params_x[-1] - mu_x) / nx
    
            gamma_x = np.sqrt(1/nx)
            sigma_x = sigma_x + gamma_x * (np.exp(log_rx) - 0.3)
                    
        iteration += 1
        
        if iteration == maximum_iteration:
            break
        
    print(iteration)

    if iteration != maximum_iteration:
        for i in range(k):
            lamb_x = Gamma_x + eps * np.eye(len(Gamma_x))
            tuning_x = np.exp(2 * sigma_x) * lamb_x

            prev_x = params_x[-1]
            
            x_prop = scipy.stats.multivariate_normal.rvs(prev_x, tuning_x)
            log_u = np.log(random.uniform(0, 1))

            
            log_rx = min(0, logpdf(x_prop) - logpdf(prev_x))
    
            if log_u <= log_rx:
                params_x.append(x_prop)
                params_y.append(x_prop)

            else:
                params_x.append(prev_x)
                params_y.append(prev_x)
        
    return params_x, params_y, iteration
    

In [15]:
def coupled_adaptive_metropolis2(data, x_init, y_init, converge_fun, coupling_fun, logpdf_fun, eps = 0.01, k = 100, lag = 1, maximum_iteration = 10000):
    params_x = [np.array(x_init)]
    params_y = [np.array(y_init)]

    mu_x = params_x[-1]
    Gamma_x = np.zeros(len(params_x[-1]))
    sigma_x = np.ones(len(params_x[-1])) * 0.7
    nx = 0
    gamma_x = 1


    mu_y = params_y[-1]
    Gamma_y = np.zeros(len(params_y[-1]))
    sigma_y = np.ones(len(params_y[-1])) * 0.7
    ny = 0
    gamma_y = 1
    
    iteration = 0
    
    while not converge_fun(params_x[-1], params_y[-1], Gamma_x, Gamma_y, sigma_x, sigma_y, eps, 0.05):
        if lag <= iteration:
            lamb_x = Gamma_x + eps
            lamb_y = Gamma_y + eps

            tuning_x = np.exp(2 * sigma_x) * lamb_x
            tuning_y = np.exp(2 * sigma_y) * lamb_y

            x_prop = np.zeros(len(params_x[-1]))
            y_prop = np.zeros(len(params_x[-1]))

            log_rx = np.zeros(len(params_x[-1]))
            log_ry = np.zeros(len(params_y[-1]))

            for j in range(len(params_x[-1])):
                prev_x = params_x[-1][j]
                prev_y = params_y[-1][j]
                
                x_rv = lambda: scipy.stats.norm.rvs(prev_x, np.sqrt(tuning_x[j]))
                y_rv = lambda: scipy.stats.norm.rvs(prev_y, np.sqrt(tuning_y[j]))
                log_x = lambda z: scipy.stats.norm.logpdf(z, prev_x, np.sqrt(tuning_x[j]))
                log_y = lambda z: scipy.stats.norm.logpdf(z, prev_y, np.sqrt(tuning_y[j]))

                temp_x, temp_y = coupling_fun(prev_x, prev_y, x_rv, y_rv, log_x, log_y, 0)

                
                logpdf_x = lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:])))
                logpdf_y = lambda z: logpdf_fun(data, np.hstack((y_prop[:j], z, params_y[-1][j + 1:])))
    
                log_u = np.log(random.uniform(0, 1))
    
                log_rx[j] = min(0, logpdf_x(temp_x) - logpdf_x(prev_x))
                log_ry[j] = min(0, logpdf_y(temp_y) - logpdf_y(prev_y))
    
                if log_u <= log_rx[j]:
                    x_prop[j] = temp_x
                else:
                    x_prop[j] = prev_x
    
                if log_u <= log_ry[j]:
                    y_prop[j] = temp_y
                else:
                    y_prop[j] = prev_y

            params_x.append(x_prop)
            params_y.append(y_prop)
    
            nx += 1
            ny += 1
    
            mx = params_x[-1] - mu_x
            Gamma_x = Gamma_x + (np.power(mx, 2) - Gamma_x) / nx
            mu_x = mu_x + (params_x[-1] - mu_x) / nx
            gamma_x = np.sqrt(1/nx)
            sigma_x = sigma_x + gamma_x * (np.exp(log_rx) - 0.3)
    
            my = params_y[-1] - mu_y
            Gamma_y = Gamma_y + (np.power(my, 2) - Gamma_y) / ny
            mu_y = mu_y + (params_y[-1] - mu_y) / ny
            gamma_y = np.sqrt(1/ny)
            sigma_y = sigma_y + gamma_y * (np.exp(log_ry) - 0.3)

    
        else:
            lamb_x = Gamma_x + eps
            tuning_x = np.exp(2 * sigma_x) * lamb_x
            
            x_prop = np.zeros(len(params_x[-1]))

            log_rx = np.zeros(len(params_x[-1]))

            for j in range(len(params_x[-1])):
                prev_x = params_x[-1][j]
                
                temp_x = scipy.stats.norm.rvs(prev_x, np.sqrt(tuning_x[j]))
                
                logpdf_x = lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:])))
    
                log_u = np.log(random.uniform(0, 1))
    
                log_rx[j] = min(0, logpdf_x(temp_x) - logpdf_x(prev_x))
    
                if log_u <= log_rx[j]:
                    x_prop[j] = temp_x
                else:
                    x_prop[j] = prev_x

            params_x.append(x_prop)
    
            nx += 1
    
            mx = params_x[-1] - mu_x
            Gamma_x = Gamma_x + (np.power(mx, 2) - Gamma_x) / nx
            mu_x = mu_x + (params_x[-1] - mu_x) / nx
            gamma_x = np.sqrt(1/nx)
            sigma_x = sigma_x + gamma_x * (np.exp(log_rx) - 0.3)
 
        iteration += 1
        
        if iteration == maximum_iteration:
            break
        
    print(iteration)
    
    if iteration != maximum_iteration:
        for i in range(k):
            lamb_x = Gamma_x + eps
            tuning_x = np.exp(2 * sigma_x) * lamb_x
            
            x_prop = np.zeros(len(params_x[-1]))

            log_rx = np.zeros(len(params_x[-1]))

            for j in range(len(params_x[-1])):
                prev_x = params_x[-1][j]
                
                temp_x = scipy.stats.norm.rvs(prev_x, np.sqrt(tuning_x[j]))
                
                logpdf_x = lambda z: logpdf_fun(data, np.hstack((x_prop[:j], z, params_x[-1][j + 1:])))
    
                log_u = np.log(random.uniform(0, 1))
    
                log_rx[j] = min(0, logpdf_x(temp_x) - logpdf_x(prev_x))
    
                if log_u <= log_rx[j]:
                    x_prop[j] = temp_x
                else:
                    x_prop[j] = prev_x

            params_x.append(x_prop)
            params_y.append(x_prop)
        
    return params_x, params_y, iteration
    